In [ ]:
!pip install ultralytics
!pip install boto3
!pip install scikit-learn
!pip install scikit-image

In [ ]:
from ultralytics import YOLO
import boto3
import os
import torch
import torchvision
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from SubModels import BollardNet, RoofRackNet
from skimage import io
from glob import glob
from pathlib import Path

print(torch.cuda.is_available()) # comment this line if no gpu available

In [ ]:
def download_data_from_s3(client, bucket_name, prefix):
    print(client.list_objects_v2(Bucket=bucket_name, Prefix=prefix))
    for obj in client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']:
        if not os.path.exists(os.path.join(os.getcwd(), os.path.dirname(obj['Key']))):
            os.makedirs(os.path.dirname(obj['Key']))
        if obj['Key'][-1] != '/':
            client.download_file(bucket_name, obj['Key'], obj['Key'])

def get_keys(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        if len(lines) != 3:
            raise Exception("Unable to read keys. Invalid file format.")
        return (lines[0][:-1], lines[1][:-1], lines[2])

id, secret, name = get_keys("aws_keys.txt")
download_data_from_s3(boto3.client('s3', aws_access_key_id=id, aws_secret_access_key=secret), name, "model")
# download_data_from_s3(boto3.client('s3', aws_access_key_id=id, aws_secret_access_key=secret), name, "submodels")

In [ ]:
labels = glob(os.path.join(os.getcwd(), "model/labels/*/*.txt"), recursive=True)


root_dir = Path(labels[0]).parent.parent.parent
for label in labels:
    label = Path(label)
    subset = label.parent.name
    img_path = [(root_dir / "images" / subset / label.with_suffix(sfx).name)
                for sfx in [".png", ".jpg", ".PNG", ".JPG"]]
    img_path = [pth for pth in img_path if pth.exists()][0]
    boxes = []
    points = []
    classes = []
    save_pth = root_dir / "labels_kp" / subset / label.name
    save_pth.parent.mkdir(parents=True, exist_ok=True)
    with open(label) as f:
        lines = f.readlines()
        for line in lines:
            splits = line.rstrip().split(" ")
            cls_id = int(splits[0])
            box = splits[1:]
            if not box:
                with open(save_pth, "w") as f:
                    pass
                continue

            box = [float(pt) for pt in box]
            point = (box[0], box[1])
            points.append(point)
            boxes.append(box)
            classes.append(cls_id)

    with open(save_pth, "w") as f:
        for point, box, cls_id in zip(points, boxes, classes):
            f.writelines(f"{cls_id} {box[0]} {box[1]} {box[2]} {box[3]} {point[0]} {point[1]} 1 \n")


In [ ]:
# version 1 of the detection model
model = YOLO('yolov8m-pose.pt')
model.to('cuda')

results = model.train(data="model/geoguessr.yaml", epochs=150,
                      imgsz=1920, rect=True, batch=4,
                      fliplr=0, 
                      mosaic=0, 
                      hsv_h=0, 
                      hsv_s=0, 
                      hsv_v=0, 
                      dropout=0.1)

In [ ]:
model_d = YOLO("ultralytics/cfg/models/v8/yolov8m.yaml").load(model.model)
model_d.ckpt = {"model": model_d.model}
model_d.save("best_detect.pt")
results = model_d.val(data="model/geoguessr.yaml")

In [ ]:
# version 2 of the detection model
model = YOLO('yolov8m.pt')
model.to('cuda') # comment this line if no gpu available

results = model.train(data="datasets/geoguessr.yaml", 
                      batch=4,
                      epochs=150,  
                      imgsz=1920, 
                      rect=True, 
                      iou=0.7, 
                      visualize=True, 
                      augment=True, 
                      fliplr=0, 
                      mosaic=0, 
                      hsv_h=0, 
                      hsv_s=0, 
                      hsv_v=0, 
                      dropout=0.1)
results = model.val()

In [ ]:
def load_data(path, batch_size = 16, val_split=0.2):
    def train_val_dataset(dataset, val_split=0.2):
        train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
        datasets = {}
        datasets['train'] = Subset(dataset, train_idx)
        datasets['val'] = Subset(dataset, val_idx)
        return datasets
    
    dataset = torchvision.datasets.ImageFolder(root=path, 
                                       transform=transforms.Compose([
                                           transforms.ToTensor()
                                       ]))

    datasets = train_val_dataset(dataset, val_split)
    dataset_train_loader = DataLoader(datasets['train'], batch_size=batch_size, shuffle=True, num_workers=8)
    dataset_val_loader = DataLoader(datasets['val'], batch_size=batch_size, shuffle=True, num_workers=8)

    return dataset_train_loader, dataset_val_loader

def train_model(model, data_path, val_split=0.2, batch_size=16, epochs=20):
    dataset_train_loader, dataset_val_loader = load_data(data_path, batch_size=batch_size, val_split=val_split)
    
    model.to('cuda')
    
    optimizer = optim.Adam(model.parameters(), lr=5e-6)
    lossFn = nn.CrossEntropyLoss()
    
    trainSteps = len(dataset_train_loader.dataset) // batch_size
    valSteps = len(dataset_val_loader.dataset) // batch_size
    
    for e in range(0, epochs):
        model.train()
        
        totalTrainLoss = 0
        totalValLoss = 0
        
        trainCorrect = 0
        valCorrect = 0
        for (x, y) in dataset_train_loader:
            (x, y) = (x.to('cuda'), y.to('cuda'))
            
            pred = model(x)
            #print(pred)
            loss = lossFn(pred, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss
            trainCorrect += (pred.argmax(1) == y).type(
    			torch.float).sum().item()
        
        with torch.no_grad():
            model.eval()
            
            for (x, y) in dataset_val_loader:
                (x, y) = (x.to('cuda'), y.to('cuda'))
                
                pred = model(x)
                totalValLoss += lossFn(pred, y)
                
                valCorrect += (pred.argmax(1) == y).type(
    				torch.float).sum().item()
    
        avgTrainLoss = totalTrainLoss / trainSteps
        avgValLoss = totalValLoss / valSteps
        
        trainCorrect = trainCorrect / len(dataset_train_loader.dataset)
        valCorrect = valCorrect / len(dataset_val_loader.dataset)
        
        print("[INFO] EPOCH: {}/{}".format(e + 1, epochs))
        print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
    		avgTrainLoss, trainCorrect))
        print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
    		avgValLoss, valCorrect))

In [ ]:
bollard_model = BollardNet()
train_model(bollard_model, 'submodels/bollard')

In [ ]:
roof_rack_model = RoofRackNet()
train_model(roof_rack_model, 'submodels/gc-roof-rack', val_split=0.5, batch_size=1, epochs=1)